In [2]:
import numpy as np

In [36]:
import FATS 
import os 
import math
from astropy.table import Table, Column, join 
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery.vizier import Vizier
#np.seterr(divide='ignore', invalid='ignore')
from __future__ import division

In [37]:
path ="/Users/swebb/Documents/for_test_michelle/test/"

In [42]:
lc_with_only_zeros = []
used_lcs = []
filenames = []

Autocor_length = []
Beyond1Std = []
CAR_sigma = []
CAR_mean = []
CAR_tau =[]
Con =[]
Eta_e = []
LinearTrend = []
MaxSlope =[]
Mean = []
Meanvariance = []
MedianAbsDev =[]
MedianBRP =[]
PairSlopeTrend =[]
PercentAmplitude =[]
Q31 =[]
Rcs =[]
Skew =[]
SlottedA_length =[]
SmallKurtosis =[]
Std = []
StetsonK_AC = []
Amplitudes = []
VariabilityIndex = []
pmra = []
pmde = []
gaia_G_RP = []
gaia_BP_G = []
detection_fraction = []


def RAsex_to_RAdec(fRAsex):
    frah = float(fRAsex[0:2])
    #print(frah)
    fram = float(fRAsex[2:4])
    #print(fram)
    fras = float(fRAsex[4:])
    #print(fras)
    #fRAdec = (frah*3600.0+fram*60.0+fras)/3600.0
    fRAdec = ((1/1 * frah) + (1/60 *fram) + (1/3600 *fras))* (360/24)
    return fRAdec

def DEsex_to_DEdec(fDEsex):
    fded = float(fDEsex[0:3])
    print(fded)
    fdem = float(fDEsex[3:5])
    print(fdem)
    fdes = float(fDEsex[5:])
    print(fdes)
    fDEdec = (math.fabs(fded)*3600.0+fdem*60.0+fdes)/3600.0
    if fDEsex[0] == '-':
        fDEdec = fDEdec * -1
    return fDEdec

for filename in os.listdir(path):
    #print i
    if filename.startswith('DWF'):
        print filename
        #print '-------'
        #print path + i 
        #print '-------'
        try: 
            mjd, mag, emag, uplim = np.loadtxt(path + filename, unpack = True, skiprows=1)
            #print mjd
        except: 
            print filename
        #mjd = np.loadtxt(path+i, usecols =1, skiprows =1)
        sum_mag = np.sum(mag)
        #print(sum_mag)
        if sum_mag == 0: 
            lc_with_only_zeros.append(filename)
        elif sum_mag != 0: 
            used_lcs.append(filename)
            
                    
            #print(flux)
            
            
            ''' Remove the non detections '''
            clean_mjd = []
            clean_mag = []
            clean_emag = []
            
            for l,m,n in zip(mjd,mag, emag):
                if m != 0:
                    if m < 25: 
                        clean_mjd.append(l)
                        clean_mag.append(m)
                        clean_emag.append(n)
                elif m == 0:
                    pass 
            
            print(clean_mjd, clean_mag, clean_emag)
            
            if len(clean_mjd) > 3 :
                
                lc = np.array([clean_mag, clean_mjd, clean_emag])
                a=FATS.FeatureSpace(featureList=['Autocor_length', 'Beyond1Std', 'CAR_sigma', 'CAR_mean', 
                                             'CAR_tau', 'Con', 'Eta_e', 'LinearTrend', 'MaxSlope', 
                                             'Mean', 'Meanvariance', 'MedianAbsDev', 'MedianBRP', 
                                             'PairSlopeTrend', 'PercentAmplitude', 'Q31', 'Rcs', 'Skew',
                                             'SlottedA_length', 'SmallKurtosis', 'Std',
                                             'StetsonK_AC' ], Data=['magnitude','time', 'error'])
                a=a.calculateFeature(lc)
                results = a.result(method='dict')
                #print results
                filenames.append(filename)
                Autocor_length.append(results['Autocor_length'])
                Beyond1Std.append(results['Beyond1Std'])
                CAR_sigma.append(results['CAR_sigma'])
                CAR_mean.append(results['CAR_mean'])
                CAR_tau.append(results['CAR_tau'])
                Con.append(results['Con'])
                Eta_e.append(results['Eta_e'])
                LinearTrend.append(results['LinearTrend'])
                MaxSlope.append(results['MaxSlope'])
                Mean.append(results['Mean'])
                Meanvariance.append(results['Meanvariance'])
                MedianAbsDev.append(results['MedianAbsDev'])
                MedianBRP.append(results['MedianBRP'])
                PairSlopeTrend.append(results['PairSlopeTrend'])
                PercentAmplitude.append(results['PercentAmplitude'])
                Q31.append(results['Q31'])
                Rcs.append(results['Rcs'])
                Skew.append(results['Skew'])
                SlottedA_length.append(results['SlottedA_length'])
                SmallKurtosis.append(results['SmallKurtosis'])
                Std.append(results['Std'])
                StetsonK_AC.append(results['StetsonK_AC'])
            
            
            
                # Find Amplitude because FATS is unable to 
                N = len(mag)
                sorted_mag = np.sort(mag)
                amp = (np.median(sorted_mag[-int(math.ceil(0.05*N)):]) - np.median(sorted_mag[0:int(math.ceil(0.05*N))])) / 2
                #print amp
                Amplitudes.append(amp)
                
                # Find VariabilityIndex beacuase FATS has it removed 
                N = len(mag)
                sigma2 = np.var(mag)
                VarIndex = 1/((N-1)*sigma2)*np.sum(np.power(mag[1:]-mag[:-1], 2))
                VariabilityIndex.append(VarIndex)
                
                # Find GAIA colors 
                    #First get the RA and DEC seperated 
                print('-------------------------------------')
                RA_h = filename[3:5]
                RA_m = filename[5:7]
                RA_s = filename[7:13]
                DEC_d = filename[13:16]
                DEC_m = filename[16:18]
                DEC_s = filename[18:24]
                print('-------------------------------------')
                
                coords = SkyCoord(RA_h + ' '+RA_m + ' '+RA_s +' '+str(DEC_d)+' '+ DEC_m +' '+ DEC_s + ' ', unit=(u.hourangle, u.deg))
                print(coords)
                RA = coords.ra.degree
                DEC = coords.dec.degree
                
                print('-------------------------------------')
                
                GAIA_DR2 = 'I/345'
                result = Vizier.query_region(str(RA)+' '+str(DEC), radius="0d0m2s", catalog=GAIA_DR2)
                #print(result) 
                try: 
                    pmra.append(float(result[0]['pmRA']))
                except: 
                    pmra.append(0)
                
                try: 
                    pmde.append(float(result[0]['pmDE']))
                except: 
                    pmde.append(0)
                    
                try: 
                    gaia_G_RP.append(float(result[0]['G-RP']))
                except:
                    gaia_G_RP.append(0)
                    
                try: 
                    gaia_BP_G.append(float(result[0]['BP-G']))
                except:
                    gaia_BP_G.append(0)
                
                    
                # find fraction of detections to non detections 
                detects = 0
                non_detects = 0
                
                for mags in mag: 
                    if mags != 0: 
                        detects += 1
                    else:
                        pass
                for uplims in uplim:
                    if uplims != 0:
                        non_detects += 1
                    else:
                        pass 
                print('detect len: ' + str(detects))
                print('number of points: '+ str(len(mag)))
                
                
                detection_fraction.append(detects/len(mag))
                
                
                
                
                    
            else: 
                print('Not enough data points')
                
            
            
            


DWF162557.862-733111.347_160726
([57595.17082153, 57595.17138714, 57595.17195109, 57595.17308919, 57595.17435267, 57595.17491524, 57595.17547297, 57595.17603672, 57595.17659504, 57595.17715042, 57595.17770744, 57595.17826043, 57595.178819, 57595.1793767, 57595.17992852, 57595.18048823, 57595.18104154, 57595.18270765, 57595.18549453, 57595.18605867, 57595.18661521, 57595.18716887, 57595.18772926], [20.836999893188477, 21.003999710083008, 21.273000717163086, 20.923999786376953, 20.964000701904297, 20.881999969482422, 21.040000915527344, 20.889999389648438, 20.804000854492188, 20.937999725341797, 21.448999404907227, 20.81399917602539, 20.860000610351562, 20.702999114990234, 20.76799964904785, 21.139999389648438, 20.704999923706055, 20.871000289916992, 20.966999053955078, 20.954999923706055, 20.756999969482422, 20.884000778198242, 20.895999908447266], [0.09600000083446503, 0.0949999988079071, 0.11500000208616257, 0.09300000220537186, 0.09700000286102295, 0.10100000351667404, 0.097999997437

/anaconda3/envs/py2.7/lib/python2.7/site-packages/astroquery/utils/commons.py:171: UserWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "


-------------------------------------
-------------------------------------
<SkyCoord (ICRS): (ra, dec) in deg
    (246.48884167, -72.80234)>
-------------------------------------
detect len: 29
number of points: 30
DWF162557.058-721038.841_160726
DWF162557.077-734117.867_160726
([57595.17082153, 57595.17138714, 57595.17195109, 57595.17308919, 57595.17435267, 57595.17491524, 57595.17547297, 57595.17603672, 57595.17659504, 57595.17715042, 57595.17770744, 57595.17826043, 57595.178819, 57595.1793767, 57595.17992852, 57595.18048823, 57595.18104154, 57595.18159692, 57595.18215122, 57595.18270765, 57595.18326585, 57595.18549453, 57595.18605867, 57595.18661521, 57595.18716887, 57595.18772926], [18.363000869750977, 18.44700050354004, 18.32200050354004, 18.136999130249023, 18.417999267578125, 18.259000778198242, 18.349000930786133, 18.23900032043457, 18.2810001373291, 18.235000610351562, 18.298999786376953, 18.336000442504883, 18.312000274658203, 18.319000244140625, 18.406999588012695, 18.32600

/anaconda3/envs/py2.7/lib/python2.7/site-packages/astroquery/utils/commons.py:171: UserWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "


-------------------------------------
-------------------------------------
<SkyCoord (ICRS): (ra, dec) in deg
    (246.48782083, -73.68829639)>
-------------------------------------
detect len: 26
number of points: 30
DWF162557.156-723125.779_160726
([57595.17082153, 57595.17138714, 57595.17195109, 57595.17252141, 57595.17308919, 57595.17435267, 57595.17491524, 57595.17547297, 57595.17603672, 57595.17659504, 57595.17715042, 57595.17770744, 57595.17826043, 57595.178819, 57595.1793767, 57595.17992852, 57595.18048823, 57595.18104154, 57595.18159692, 57595.18215122, 57595.18270765, 57595.18326585, 57595.18382406, 57595.18437732, 57595.18493814, 57595.18549453, 57595.18605867, 57595.18661521, 57595.18716887, 57595.18772926], [19.55500030517578, 19.652999877929688, 19.750999450683594, 19.70199966430664, 19.618999481201172, 19.6299991607666, 19.625999450683594, 19.545000076293945, 19.621999740600586, 19.665000915527344, 19.733999252319336, 19.61199951171875, 19.635000228881836, 19.6790008544

...

/anaconda3/envs/py2.7/lib/python2.7/site-packages/astroquery/utils/commons.py:171: UserWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "
/anaconda3/envs/py2.7/lib/python2.7/site-packages/astroquery/utils/commons.py:171: UserWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "
/anaconda3/envs/py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:172: UserWarning: Warning: converting a masked element to nan.
/anaconda3/envs/py2.7/lib/python2.7/site-packages/ipykernel_launcher.py:177: UserWarning: Warning: converting a masked element to nan.


-------------------------------------
-------------------------------------
<SkyCoord (ICRS): (ra, dec) in deg
    (246.49059583, -73.54400861)>
-------------------------------------
detect len: 25
number of points: 30
DWF162557.945-720831.629_160726
DWF162557.966-733210.325_160726
DWF162557.003-723305.774_160726
([57595.17082153, 57595.17138714, 57595.17195109, 57595.17252141, 57595.17308919, 57595.17435267, 57595.17491524, 57595.17547297, 57595.17603672, 57595.17659504, 57595.17715042, 57595.17770744, 57595.17826043, 57595.178819, 57595.1793767, 57595.17992852, 57595.18048823, 57595.18104154, 57595.18159692, 57595.18215122, 57595.18270765, 57595.18326585, 57595.18382406, 57595.18437732, 57595.18493814, 57595.18549453, 57595.18605867, 57595.18661521, 57595.18716887, 57595.18772926], [19.663000106811523, 19.55900001525879, 19.670000076293945, 19.64299964904785, 19.54400062561035, 19.611000061035156, 19.617000579833984, 19.50200080871582, 19.56100082397461, 19.54400062561035, 19.5919990

/anaconda3/envs/py2.7/lib/python2.7/site-packages/astroquery/utils/commons.py:171: UserWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "
/anaconda3/envs/py2.7/lib/python2.7/site-packages/astroquery/utils/commons.py:171: UserWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "


detect len: 30
number of points: 30
DWF162557.048-721110.730_160726
([57595.17082153, 57595.17308919, 57595.17435267, 57595.17547297, 57595.17603672, 57595.17659504, 57595.17715042, 57595.17770744, 57595.17826043, 57595.178819, 57595.1793767, 57595.17992852, 57595.18048823, 57595.18104154, 57595.18159692, 57595.18215122, 57595.18326585, 57595.18549453, 57595.18661521, 57595.18716887], [18.663999557495117, 19.03700065612793, 18.207000732421875, 18.226999282836914, 18.312999725341797, 18.14699935913086, 18.158000946044922, 18.243999481201172, 18.270999908447266, 18.17099952697754, 18.118000030517578, 18.284000396728516, 18.072999954223633, 18.16200065612793, 18.267000198364258, 18.32699966430664, 18.142000198364258, 18.343000411987305, 18.211000442504883, 18.156999588012695], [0.017000000923871994, 0.024000000208616257, 0.017999999225139618, 0.017999999225139618, 0.01899999938905239, 0.01600000075995922, 0.017000000923871994, 0.017999999225139618, 0.01899999938905239, 0.01700000092387199

/anaconda3/envs/py2.7/lib/python2.7/site-packages/astroquery/utils/commons.py:171: UserWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "


In [41]:
print 'Not used'
print len(lc_with_only_zeros) 
print 'Used'
print len(used_lcs)

feature_table = Table()
feature_table['LC_name'] = filenames
feature_table['Autocor_length'] = Autocor_length
feature_table['Beyond1Std'] = Beyond1Std
feature_table['CAR_sigma'] = CAR_sigma
feature_table['CAR_mean'] = CAR_mean
feature_table['CAR_tau'] = CAR_tau
feature_table['Con'] = Con
feature_table['Eta_e'] = Eta_e
feature_table['LinearTrend'] = LinearTrend
feature_table['MaxSlope'] = MaxSlope
feature_table['Mean'] =Mean
feature_table['Meanvariance'] =Meanvariance
feature_table['MedianAbsDev'] =MedianAbsDev
feature_table['MedianBRP'] =MedianBRP
feature_table['PairSlopeTrend'] =PairSlopeTrend
feature_table['PercentAmplitude'] =PercentAmplitude
feature_table['Q31'] =Q31
feature_table['Rcs'] =Rcs
feature_table['Skew'] =Skew
feature_table['SlottedA_length'] =SlottedA_length
feature_table['SmallKurtosis'] =SmallKurtosis
feature_table['Std'] =Std
feature_table['StetsonK_AC'] =StetsonK_AC
feature_table['Amplitudes'] = Amplitudes
feature_table['VariabilityIndex'] = VariabilityIndex
feature_table['pmra'] = pmra 
feature_table['pmde'] = pmde 
feature_table['gaia_G_RP'] = gaia_G_RP
feature_table['gaia_BP_G'] = gaia_BP_G
feature_table['detection_fraction'] = detection_fraction

print feature_table['detection_fraction']
output = '/Users/swebb/Documents/for_test_michelle/test.ascii'
feature_table.write(output, format = 'ascii', overwrite = True)

Not used
7
Used
11
 detection_fraction
-------------------
 0.7666666666666667
 0.9666666666666667
 0.8666666666666667
                1.0
 0.8666666666666667
 0.8666666666666667
0.43333333333333335
 0.8333333333333334
                1.0
 0.6666666666666666


In [27]:
filename = 'DWF164557.862'
RAsex_to_RAdec(filename[3:13])

240.0

In [40]:
def RAsex_to_RAdec(fRAsex):
    frah = float(fRAsex[0:2])
    #print(frah)
    fram = float(fRAsex[2:4])
    #print(fram)
    fras = float(fRAsex[4:])
    #print(fras)
    #fRAdec = (frah*3600.0+fram*60.0+fras)/3600.0
    fRAdec = ((1/1 * 16) + (1/60 *5) + (1/3600 *57.867))* (360/24)
    return fRAdec

In [41]:
filename = 'DWF160557.862'
RAsex_to_RAdec(filename[3:13])

240.0